# Definición parámetros

In [ ]:
# === HASUMI — Percentiles de Amount (S3) y Factor (S3/AVG177) — sin 180d ======
import pandas as pd
import numpy as np
import math

pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# -------- Parámetros --------
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"     # <-- ajusta la ruta
SUBSUBSEGMENTS = ["I-1", "I-2"]                     # <-- ajusta el sub-subsegmento
AMOUNT_QS = [0.85, 0.90, 0.95, 0.97, 0.99]
FACTOR_QS = [0.95, 0.97, 0.99]

# -------- Carga + filtro HASUMI --------
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()
df["tx_type"]        = df["tx_type"].astype(str).str.title()

if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

mask = (
    df["tx_direction"].eq("Inbound") &
    df["tx_type"].eq("Cash") &
    df["tx_date_time"].notna() &
    df["tx_base_amount"].notna()
)
g = df.loc[mask, ["customer_id","tx_date_time","tx_base_amount"]].copy()

if g.empty:
    print("No hay transacciones elegibles para HASUMI.")
else:
    amt_points = []
    fac_points = []

    for cid, sub in g.groupby("customer_id", sort=False):
        sub = sub.sort_values("tx_date_time")
        # Suma diaria y ventanas
        daily = sub.set_index("tx_date_time")["tx_base_amount"].abs().resample("D").sum()
        if daily.empty:
            continue

        S3 = daily.rolling("3D").sum()                                   # suma en 3 días
        AVG177 = S3.shift(3).rolling("177D", min_periods=1).mean()        # promedio histórico disponible

        # Amount: S3 > 0
        ok_amt = (S3 > 0) & S3.notna()
        if ok_amt.any():
            amt_points.append(S3.loc[ok_amt])

        # Factor: S3 > 0 y AVG177 > 0
        ok_fac = ok_amt & (AVG177 > 0) & AVG177.notna()
        if ok_fac.any():
            fac_points.append((S3.loc[ok_fac] / AVG177.loc[ok_fac]).replace([np.inf, -np.inf], np.nan).dropna())

    S_amt = pd.concat(amt_points) if len(amt_points) else pd.Series(dtype=float)
    S_fac = pd.concat(fac_points) if len(fac_points) else pd.Series(dtype=float)

    amount_q = S_amt.quantile(AMOUNT_QS) if len(S_amt) else pd.Series(index=AMOUNT_QS, dtype=float)
    factor_q = S_fac.quantile(FACTOR_QS) if len(S_fac) else pd.Series(index=FACTOR_QS, dtype=float)

    df_amount = pd.DataFrame({
        "percentil": [f"p{int(q*100)}" for q in AMOUNT_QS],
        "Amount": [amount_q.get(q, np.nan) for q in AMOUNT_QS]
    })

    df_factor = pd.DataFrame({
        "percentil":        [f"p{int(q*100)}" for q in FACTOR_QS],
        "Factor_raw":       [factor_q.get(q, np.nan) for q in FACTOR_QS],
        "Factor_recommend": [int(math.ceil(factor_q.get(q))) if pd.notna(factor_q.get(q, np.nan)) and np.isfinite(factor_q.get(q)) else np.nan
                             for q in FACTOR_QS]
    })

    print("=== HASUMI — Resultados (sin requisito de 180 días) ===")
    print(f"Puntos Amount (S3): {len(S_amt):,} | Puntos Factor (S3/AVG177): {len(S_fac):,}\n")
    print("— Amount (S3 en 3 días, CLP) —")
    display(df_amount)
    print("— Factor (S3 / AVG177) —")
    display(df_factor)


C:\Users\vinso\AppData\Local\Temp\ipykernel_10428\1385000235.py:15: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== HASUMI — Resultados (sin requisito de 180 días) ===
Puntos Amount (S3): 1,116 | Puntos Factor (S3/AVG177): 1,062

— Amount (S3 en 3 días, CLP) —


,percentil,Amount
0,p85,"1,879,340,000.00"
1,p90,"2,375,400,000.00"
2,p95,"3,681,800,000.00"
3,p97,"4,757,501,445.00"
4,p99,"7,163,761,747.00"


— Factor (S3 / AVG177) —


,percentil,Factor_raw,Factor_recommend
0,p95,9.61,10
1,p97,11.61,12
2,p99,37.69,38


# Simulación alertas

In [ ]:
# === HASUMI — Sensibilidad (Actual vs propuestos) ==============================
# LÓGICA EXACTA (sin exigir “historia >180d”):
#   tx_direction = Inbound
#   AND sum base en 3d (S3) > [Amount]
#   AND S3 > [Factor] * { [sum en 180d (S180) - S3] / 59 }     # promedio 3d histórico de los 177 días previos
# Notas:
# - Ventanas móviles por cliente que incluyen el día actual (rolling "3D" y "180D").
# - No imponemos el requisito “count all-time > count en 180d” porque la BBDD aún no tiene 180 días.
# - Se usa |tx_base_amount| para evitar signos contables.
# Unidad = ventanas cliente–día

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# ---- EDITA AQUÍ ---------------------------------------------------------------
PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = "I-1"               # <-- ajusta el sub-subsegmento
PARAMS = {
    "Actual": {"Amount": 16_000_000, "Factor": 169},
    "p95":    {"Amount": 888_000_000, "Factor": 748},
    "p97":    {"Amount": 957_000_000, "Factor": 2491},
    "p99":    {"Amount": 1_407_000_293, "Factor": 12329},
}
# -------------------------------------------------------------------------------

# --- Carga + filtro Inbound Cash ---
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()
df["tx_type"]        = df["tx_type"].astype(str).str.title()
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

mask = (
    df["tx_direction"].eq("Inbound") &
    df["tx_type"].eq("Cash") &
    df["customer_id"].notna() &
    df["tx_date_time"].notna() &
    df["tx_base_amount"].notna()
)
g = df.loc[mask, ["customer_id","tx_date_time","tx_base_amount"]].copy()

if g.empty:
    print("No hay transacciones elegibles para HASUMI.")
else:
    # Construir métricas diarias por cliente: S3 y S180 (con montos en valor absoluto)
    parts = []
    for cid, sub in g.groupby("customer_id", sort=False):
        daily = (sub.set_index("tx_date_time")["tx_base_amount"].abs()
                    .resample("D").sum())
        if daily.empty:
            continue
        S3   = daily.rolling("3D").sum()
        S180 = daily.rolling("180D").sum()
        parts.append(pd.DataFrame({
            "customer_id": cid,
            "date": daily.index,
            "S3": S3.values,
            "S180": S180.values
        }))

    M = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["customer_id","date","S3","S180"])

    if M.empty:
        print("No se pudieron construir ventanas 3d (M vacío).")
    else:
        # Promedio 3d histórico de los 177 días previos
        M["avg3_hist"] = (M["S180"] - M["S3"]) / 59.0

        # --- Parámetros (visualización) ---
        order = ["Actual","p85","p90","p95","p97","p99"]
        if PARAMS:
            param_tbl = (pd.DataFrame(PARAMS).T
                           .loc[[k for k in order if k in PARAMS] or list(PARAMS.keys())]
                           .rename_axis("escenario")
                           .reset_index())
            print("=== HASUMI — Parámetros (Amount, Factor) ==="); display(param_tbl)
        else:
            print("⚠️  Define PARAMS (Amount, Factor) antes de ejecutar conteos.")

        # --- Conteo de alertas por escenario ---
        counts = {}
        for k, v in PARAMS.items():
            A = float(v["Amount"])
            F = float(v["Factor"])
            m = (
                (M["S3"] > A) &
                (M["avg3_hist"] > 0) &
                (M["S3"] > F * M["avg3_hist"])
            )
            counts[k] = int(M.loc[m, ["customer_id","date"]].drop_duplicates().shape[0])

        if PARAMS:
            alerts_wide = pd.DataFrame([{f"alertas_{name.lower()}": counts.get(name, 0)
                                         for name in ([k for k in order if k in PARAMS] or list(PARAMS.keys()))}])
            print("=== HASUMI — Alertas por escenario (ventanas cliente–día) ==="); display(alerts_wide)


=== HASUMI — Parámetros (Amount, Factor) ===


,escenario,Amount,Factor
0,Actual,16000000,169
1,p95,888000000,748
2,p97,957000000,2491
3,p99,1407000293,12329


=== HASUMI — Alertas por escenario (ventanas cliente–día) ===


,alertas_actual,alertas_p95,alertas_p97,alertas_p99
0,33,7,3,0
